# Module 2: Simple Neural Networks


## Part 1: Single Hidden Layer with Sigmoid Activation Function

<!-- insert png of nn1 -->
![image](nn1.png)

### #1: Illustrate the Matrices Associated with the Neural Network  

 
$X = 
\begin{bmatrix}
x_1\\
x_2\\
x_3
\end{bmatrix}$

$W1 = 
\begin{bmatrix}
w_{11} & w_{21} & w_{31}\\
w_{12} & w_{22} & w_{32}\\
w_{13} & w_{23} & w_{33}\\
w_{14} & w_{24} & w_{34}\\
\end{bmatrix}$

$H = 
\begin{bmatrix}
h_1\\
h_2\\ 
h_3\\
h_4\\
\end{bmatrix}$

$W2 =
\begin{bmatrix}
w_{11} & w_{21} & w_{31} & w_{41}\\
\end{bmatrix}$

$B = 
\begin{bmatrix}
b_1\\
b_2\\
b_3\\
b_4\\
\end{bmatrix}$

$C \in \mathbb{R}$ (c is a scalar)

$Z1 = W1 * X + B = 
\begin{bmatrix}
z_1\\
z_2\\
z_3\\
z_4\\
\end{bmatrix}$

$\sigma (z) = \frac{1}{1+e^{-z}}$

$H = \sigma(Z1)$ 

$Z2 = W2 * H + C$ (scalar)

$\hat{y} = \sigma(Z2)$ (scalar)

$y \in \mathbb{R}$ (scalar)

$\hat{y} - y \in \mathbb{R} $ (scalar)

Assuming binary cross entropy:  

$L = -y\log(\hat{y}) - (1-y)\log(1-\hat{y})$ (scalar)

### #2: Use Python to Code the Feed Forward portion of this Neural Network
a-c. Create a dataset and read it in

In [4]:
#2a-c: Create and read in dataa
import pandas as pd
import numpy as np

def read_data(path):
    return pd.read_csv(path)

def clean_data(df):
   
    nrows = df.shape[0]
    ncols = df.shape[1]
   
    x = np.empty((nrows,ncols-1))
    y = np.empty((nrows,1))

    # separate labels
    if 'LABEL' in df.columns:
        y = np.array(df['LABEL'])
        x = np.array(df[df.columns.difference(['LABEL'])])
    elif 'label' in df.columns:
        y = np.array(df['label'])
        x = np.array(df[df.columns.difference(['label'])])
    else:
        raise Exception('There is no column with the title label in the data')
    
    y = convert_labels(y)
    y = y.reshape(-1, 1) 
    x = normalize(x)
    return x, y

def convert_labels(y):
    y_values = set(y)
    y_value_0 = list(y_values)[0]
    y_value_1 = list(y_values)[1]

    y[y == y_value_0] = 0
    y[y == y_value_1] = 1
    return y

def normalize(x):
    xrows = x.shape[0]
    xcols = x.shape[1]
    xnorm = np.empty((xrows, xcols))
    for j, column in enumerate(x.T):
        max = column.max()
        min = column.min()
        for i, item in enumerate(column):
            xnorm[i,j] = (item - min)/(max - min)
    return xnorm

df = read_data('A2_Data_ILG.csv')
x, y = clean_data(df)

print('The data is: \n', df)
print('\nX is: \n', x)
print('\nY is: \n', y)

The data is: 
    experience  skill  athleticism label
0           5      8            5  good
1           4      4            3  good
2           2      2            4   bad
3           0      2            3   bad
4           5      6            5  good
5           1      0            0   bad

X is: 
 [[1.   1.   1.  ]
 [0.6  0.8  0.5 ]
 [0.8  0.4  0.25]
 [0.6  0.   0.25]
 [1.   1.   0.75]
 [0.   0.2  0.  ]]

Y is: 
 [[0]
 [0]
 [1]
 [1]
 [0]
 [1]]


In [22]:
#2d: Code a sigmoid function and the derivative
def sigmoid(z):
        return 1/(1+np.exp(-z))

def sigmoid_derivative(z):
    return sigmoid(z) * (1-sigmoid(z))

#2e: Code the loss function
def mse(y, y_hat):
    return np.square(y - y_hat).mean()

#2: Code the feedforward function
import numpy as np

class NeuralNetwork:

    def __init__(self, learning_rate, epochs):
        # shape parameters
        # hidden_layers = 1
        hidden_layer_size = 4
        output_layer_size = 1
        input_layer_size = 3

        # initialize
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.w1 = np.ones((hidden_layer_size, input_layer_size))
        self.b = np.zeros((hidden_layer_size, 1))
        self.w2 = 2 * np.ones((output_layer_size, hidden_layer_size))
        self.c = -1 * np.ones((output_layer_size, 1))

        # print w1, b, w2, c
        # print('W1 is: \n', self.w1)
        # print('\nB is: \n', self.b)
        # print('\nW2 is: \n', self.w2)
        # print('\nC is: \n', self.c)

    def feed_forward(self, x):
        self.z1 = x @ self.w1.T + self.b.T
        # print('\nZ1 is: \n', self.z1)
        self.h = sigmoid(self.z1)
        # print('\nH is: \n', self.h)
        self.z2 = self.h @ self.w2.T + self.c.T
        self.y_hat = sigmoid(self.z2)
        return self.y_hat
   
    def calculate_loss(self, y):
        self.loss = mse(y, self.y_hat)
        return self.loss

nn1 = NeuralNetwork(0.1, 1)
y_hat = nn1.feed_forward(x)
loss = nn1.calculate_loss(y)

# Print out X, y, W1, B, Z1, H1, W2, Z2, C, y^, y^-y, and L.
print('X is: \n', x)
print('\ny is: \n', y)
print('\nW1 is: \n', nn1.w1)
print('\nB is: \n', nn1.b)
print('\nZ1 is: \n', nn1.z1)
print('\nH is: \n', nn1.h)
print('\nW2 is: \n', nn1.w2)
print('\nZ2 is: \n', nn1.z2)
print('\nC is: \n', nn1.c)
print('\ny^ is: \n', nn1.y_hat)
print('\ny^-y is: \n', nn1.y_hat - y)
print('\nL is: \n', nn1.loss)

X is: 
 [[1.   1.   1.  ]
 [0.6  0.8  0.5 ]
 [0.8  0.4  0.25]
 [0.6  0.   0.25]
 [1.   1.   0.75]
 [0.   0.2  0.  ]]

y is: 
 [[0]
 [0]
 [1]
 [1]
 [0]
 [1]]

W1 is: 
 [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]

B is: 
 [[0.]
 [0.]
 [0.]
 [0.]]

Z1 is: 
 [[3.   3.   3.   3.  ]
 [1.9  1.9  1.9  1.9 ]
 [1.45 1.45 1.45 1.45]
 [0.85 0.85 0.85 0.85]
 [2.75 2.75 2.75 2.75]
 [0.2  0.2  0.2  0.2 ]]

H is: 
 [[0.95257413 0.95257413 0.95257413 0.95257413]
 [0.86989153 0.86989153 0.86989153 0.86989153]
 [0.80999843 0.80999843 0.80999843 0.80999843]
 [0.70056714 0.70056714 0.70056714 0.70056714]
 [0.93991335 0.93991335 0.93991335 0.93991335]
 [0.549834   0.549834   0.549834   0.549834  ]]

W2 is: 
 [[2. 2. 2. 2.]]

Z2 is: 
 [[6.62059301]
 [5.95913221]
 [5.47998747]
 [4.60453714]
 [6.5193068 ]
 [3.39867198]]

C is: 
 [[-1.]]

y^ is: 
 [[0.99866913]
 [0.9974245 ]
 [0.99584793]
 [0.9900928 ]
 [0.99852748]
 [0.96766301]]

y^-y is: 
 [[0.9986691331309677]
 [0.9974244985574349]
 [-0.004152070364944

In [ ]:
    def update_weights(self, y):
        print('updating weights')

    def train(self, x, y):
        self.initialize_weights()
        print('training the model.')
        for _ in range(self.epochs):
            self.feed_forward()
            self.calculate_loss(y)
            self.update_weights(y)
            return 